## ПР3

#### Подготовил Вадим Пацев группа ИЭоз-60-18

In [123]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import t, f

import warnings
warnings.filterwarnings('ignore')

In [124]:
# Данные про шахты

df = pd.DataFrame(data={'x1': [8,11,12,9,8,8,9,9,8,12], 'x2': [5,8,8,5,7,8,6,4,5,7], 'y':[5,10,10,7,5,6,6,5,6,8]})
df

,x1,x2,y
0,8,5,5
1,11,8,10
2,12,8,10
3,9,5,7
4,8,7,5
5,8,8,6
6,9,6,6
7,9,4,5
8,8,5,6
9,12,7,8


In [125]:
x = df[['x1', 'x2']]
y = df['y']

### 1. Определить параметры линейной регрессии. Пояснить их смысл.

In [126]:
est = sm.OLS(y, sm.add_constant(x)).fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     15.08
Date:                Mon, 30 May 2022   Prob (F-statistic):            0.00290
Time:                        19:33:54   Log-Likelihood:                -11.903
No. Observations:                  10   AIC:                             29.81
Df Residuals:                       7   BIC:                             30.71
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5393      1.907     -1.856      0.1

In [127]:
y_pred = est.params[0]+ est.params[1] * x['x1'] + est.params[2] * x['x2'] 
print("Оценка коэфициента b0=", est.params[0])
print("Оценка коэфициента b1=", est.params[1])
print("Оценка коэфициента b2=", est.params[2])
print("Коэффициент определения R^2=", est.rsquared)
print("Стандартное значения ошибок коэфициента s_b0=", est.bse['const'])
print("Стандартное значения ошибок коэфициента s_b1=", est.bse['x1'])
print("Стандартное значения ошибок коэфициента s_b2=", est.bse['x2'])
print("F-статистика", est.fvalue)
print("Степени свободы k2=", est.df_resid)
print("Стандартная ошибка возмущений (SEE) s=", (np.sum((y - y_pred)**2)/(est.df_resid))**0.5)

Оценка коэфициента b0= -3.5393258426966296
Оценка коэфициента b1= 0.8539325842696636
Оценка коэфициента b2= 0.3670411985018725
Коэффициент определения R^2= 0.8116194043160334
Стандартное значения ошибок коэфициента s_b0= 1.9065807170473963
Стандартное значения ошибок коэфициента s_b1= 0.22050430957080328
Стандартное значения ошибок коэфициента s_b2= 0.24294835693466965
F-статистика 15.07940828402366
Степени свободы k2= 7.0
Стандартная ошибка возмущений (SEE) s= 0.9509084389587876


In [128]:
#Вычислим Qe (SSE) — сумма квадратов оценок ошибок
sse = np.sum((y_pred - y)**2)
print("Qe = ", sse)

#Вычислим Qr (SSR) — остаточная сумма квадратов (SSR)
ssr = np.sum((y_pred - y.mean())**2)
print("Qr = ", ssr)

#calculate Q (SST)
sst = ssr + sse
print("Q = ", sst)

Qe =  6.329588014981269
Qr =  27.27041198501876
Q =  33.60000000000003


Так как Qr>Qe, то уравнение регрессии значимо.

In [129]:
print("F-статистика ", est.fvalue)

F-статистика  15.07940828402366


In [130]:
f_val = f.ppf(q=1-.05, dfn=est.df_model, dfd=est.df_resid)
print("Критическое значение (порог) F-статистики: f(alfa=0,05;k1=p=2;k2=n-p-1=7)=", f_val)

Критическое значение (порог) F-статистики: f(alfa=0,05;k1=p=2;k2=n-p-1=7)= 4.73741412777588


Так как статистика F больше порога F>f(…),  то уравнение значимо.

### 2.   Определить стандартизированные коэффициенты регрессии и коэффициенты эластичности. Пояснить их смысл.

In [131]:
b1_ = est.params[1] * np.std(x['x1'])/np.std(y)
b2_ = est.params[2] * np.std(x['x2'])/np.std(y)
e1 = est.params[1] * np.mean(x['x1'])/np.mean(y)
e2 = est.params[2] * np.mean(x['x2'])/np.mean(y)

In [132]:
print('Коэффициенты регрессии x1', b1_)
print('Коэффициенты регрессии x2', b2_)
print('Коэффициенты эластичности e1', e1)
print('Коэффициенты эластичности e2', e2)

Коэффициенты регрессии x1 0.7276941106373302
Коэффициенты регрессии x2 0.2838854958688276
Коэффициенты эластичности e1 1.1804362194315936
Коэффициенты эластичности e2 0.3400528750826172


Анализируя полученные значения, можно сделать следующие выводы. 
Увеличение только мощности пласта на одно значение s_x1 приводит к возрастанию добычи угля на 1 рабочего в среднем на 0,728 (s_y)
увеличение только уровня механизации работ на одно значение sx2 приводит к возрастанию добычи угля на 1 рабочего в среднем на 0,285 (s_y).
Увеличение первого фактора на 1% (от среднего значения) приводит в
среднем к росту добычи угля на 1,18%, а увеличение второго фактора на 1% приводит к росту добычи на 0,34%.
Таким образом, величина добычи угля сильнее зависит от мощности пла- ста, чем от уровня механизации работ. 


### 3,4. Сравнить значения коэффициента детерминации с ситуацией р=1 (см. работу №1). 

In [133]:
print('ПР1 p=', 1, 'R^2=', 0.75019)

ПР1 p= 1 R^2= 0.75019


In [134]:
print('ПР2 p=', 2, 'R^2=', est.rsquared)

ПР2 p= 2 R^2= 0.8116194043160334


In [135]:
print('ПР1 R^2^=', 1-(1-0.75019)*(len(y)-1)/(len(y)-1-1))

ПР1 R^2^= 0.71896375


In [136]:
print('ПР2 R^2^=', 1-(1-est.rsquared)*(len(y)-1)/(len(y)-2-1))

ПР2 R^2^= 0.7577963769777573


Такое увеличение (R^2) норм (R^2 с крышкой) , скорее всего, слишком мало для того, чтобы
говорить об улучшении качества модели при включении в нее второго фактора.

### 5. Проверить гипотезу о незначимости коэффициентов уравнения регрессии для уровня значимости 0,05. 

In [137]:
print('T1=', est.params[1]/est.bse['x1'])
print('T2=', est.params[2]/est.bse['x2'])

T1= 3.872634443888129
T2= 1.5107786820742821


In [138]:
t_val =abs(t.ppf(0.05/2, est.df_resid))

In [139]:
print('Критическое значение (порог) T-статистики: t(alfa=0,05;k2=7)=', t_val)

Критическое значение (порог) T-статистики: t(alfa=0,05;k2=7)= 2.3646242510102997


Проверяя неравенство |T|>t(…), получаем: первый фактор (X1 мощность пласта) значим, а второй (X2 уровень механизации) незначим. Этот результат подтверждает выводы предыдущих параграфов.

### 6.   Найти с надежностью 0,95 интервальные оценки коэффициентов  регрессии  и дисперсии возмущений.

In [140]:
t_v1 = est.params[1] + t_val * est.bse['x1']
t_n1 =est.params[1] - t_val * est.bse['x1']
t_v2 = est.params[2] + t_val * est.bse['x2']
t_n2 =est.params[2] - t_val * est.bse['x2']
print("Порог Т-статистики t(alfa=0.05, k=k2=7)=", t_val)
print("Верхняя граница Т-статистики x1=", t_v1)
print("Нижняя граница Т-статистики x1=", t_n1)
print("Верхняя граница Т-статистики x2=", t_v2)
print("Нижняя граница Т-статистики x2=", t_n2)

Порог Т-статистики t(alfa=0.05, k=k2=7)= 2.3646242510102997
Верхняя граница Т-статистики x1= 1.3753424221330675
Нижняя граница Т-статистики x1= 0.33252274640625956
Верхняя граница Т-статистики x2= 0.9415227750526987
Нижняя граница Т-статистики x2= -0.2074403780489537
